In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.db_parser import db_parser 
from lib.picklist_recommender import picklist_recommender 
from lib.question_type_recommender import question_type_recommender
from lib.script_generator import script_generator   
from lib.dbupdate_parser import dbupdate_parser
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords  

ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

ps=PorterStemmer()  
sw=stopwords.words('english')

In [ ]:
all=''
for aspx in [f for f in listdir(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2023') if re.search('aspx$', f)]:  
    s=''
    fname=fr"C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\HVA\2023\{aspx}"
    with open(fname,encoding="utf8", errors='ignore' ) as f:
        all=f'{all}\n <!-- path:{fname}  --> \n{f.read()} '  
with open(f"{ctx.get_dest()}script.aspx", 'w', encoding='UTF-8') as f:
    f.write(all)

def db_updates():
    pkforms={}
    script = ''
    for filename in os.listdir(fr'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\SA'):
        f = os.path.join(fr'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\SA', filename) 
        if not 'Import' in f: 
            sql=''
            with open(f, 'r', encoding='utf-8') as file: 
                sql=file.read()
                group = re.search("'(2023-A-\w{3,})'",sql)
                pkforms[group.groups(1)[0]]=f 
                script=f'{script}\n{"--"}path:{f}\n {sql}' 
    with open(ctx.get_dest() + '\dbupdates.sql', 'w', encoding='UTF-8') as file: 
        file.write(script)
    return pkforms
pkforms = db_updates()  

In [ ]:
files = [f for f in listdir(r'C:\Users\timko\Downloads\Questionnaire\1.5') if re.search('xlsx', f)]   
files = { 
'2023-A-HVAARC':'HVA Network Architecture Questionnaire.xlsx' ,
'2023-A-HVAASSET':'HVA Asset Management Questionnaire.xlsx' ,
'2023-A-HVABC':'HVA Business Continuity Management Questionnaire.xlsx' ,
'2023-A-HVADPRO':'HVA Data Protection Questionnaire.xlsx',
'2023-A-HVAENDPRO':'HVA Endpoint Protection Questionnaire.xlsx',
'2023-A-HVAIDENT':'HVA Identity and Access Management Questionnaire.xlsx',
'2023-A-HVAMD':'HVA Monitoring and Detection Questionnaire.xlsx',
'2023-A-HVAREM':'HVA Remote Access Questionnaire.xlsx',
'2023-A-HVAVULN':'HVA Vulnerability Management Questionnaire.xlsx',
'2023-A-HVAAPPSEC':'HVA Application Security Questionnaire.xlsx'
}  

In [ ]:
def ufencoder (s, length=10): 
    s = ''.join([ps.stem(w) for w in s.split(' ') if w.lower() not in sw]) 
    s=s.upper().strip() 
    s=re.sub('[^A-Z0-9]','',s)
    for c in 'IOU':
        if len(s) > length:
            s=re.sub(c,'',s) 
    if len(s) > length: 
        s=s[:7]+s[len(s)-3:]
    return s[:length]
    
def qnormalize(s): 
    s=s.strip()
    s=re.sub('[^A-Za-z0-9]','',str(s)) 
    return s
def normalize(s): 
    s=s.upper().strip()
    s=re.sub('[^A-Z0-9\s]','',str(s))
    s=re.sub('\s{1,}','',str(s))
    return s
normalize('Jump host access (e.g., the user accesses the HVA from an intermediate host) ')
 
def get_recommender(PK_FORM, normalizer=normalize):
    pl=sql_todf(f"""
    SELECT LTRIM(RTRIM(DisplayValue))  [Selections], PK_PickList [ML_Value], PK_PickListType [PK_PickListType]
    FROM vwPickLists  
    WHERE PK_PickListType  > 400  
    ORDER BY PK_PickListType DESC
    """, connstr)   
    pl['SelectionsNorm'] = pl['Selections'].apply(normalize) 
    pr = picklist_recommender(ctx, picklist_where=f" PK_PickListType  > 400 ") 
 
    return pr, pl

def xl_to_df(PK_FORM, version=''): 
    path=r'C:\Users\timko\Downloads\Questionnaire\\'+version + files[PK_FORM]
    print(PK_FORM + ' ' + path)
    converts={
        'Selections':lambda x: re.sub("^(\s?\d{,2}\.)",'',str(x)).strip() 
        , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip() 
        , 'ML Scored':lambda x: str(x).strip() 
        , 'Questions':lambda x: x.replace(0, None).strip() 
    }
    df=pd.read_excel(f'{path}', converters=converts,  sheet_name=4, header=2, usecols='B:I, M:R').reset_index()  
    
    for c in ['ID','Question','SML']:  df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
    for c in df.columns:  df = df.rename({c: re.sub('\s','_',c) }, axis='columns')
    
    if 'ML' in df.columns:
        df["ML_Scored"]= df["ML_Score"].apply(lambda x: str(x).strip())
        df['ML_Score']=df["ML"].apply(lambda x: str(x).strip())
        df=df.drop(columns='ML')
 
    df['PK_FORM']=PK_FORM
    df["ID"].fillna(method='ffill', inplace=True) 
    df['Question'].fillna(method='ffill', inplace=True) 
    df.drop(columns=['Check_Answer', 'index'], inplace=True)
     
    df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]   
    #df = df.loc[ df["SML"].str.contains('^SML(\d)$', na=False, regex=True) ]   
    df=df.dropna(subset = ['Selections'])
    
    df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
    df['Selections']=df['Selections'].apply(lambda s: str(s).replace("'",'`'))
    df['SelectionsNorm'] = df['Selections'].apply(normalize)
    df['nQuestion'] = df['Question'].apply(qnormalize)
    df['SML']=df['ML_Score'].apply(lambda s: f'SML{s}') 
    
    df.fillna('', inplace=True) 
    for c in df.columns:  
        df[c]=df[c].astype('str')    
    return df

def apply_picklists(df): 
    PK_FORM=df.iloc[0]['PK_FORM']
    pr, pl=get_recommender(PK_FORM=PK_FORM, normalizer=normalize)
    dfp=df.groupby(['ID'], as_index=False).agg({'SelectionsNorm':list})  
    dfp['PK_PickListType']=dfp['SelectionsNorm'].apply(lambda s: pr.recommend(s, normalizer=normalize, threshhold=(.97, .93))['PK_PicklistType'])
  
    dff=pd.merge(df,dfp.loc[:,['ID','PK_PickListType']], how='left', left_on='ID', right_on='ID')
    dff=pd.merge(dff,pl, how='left', left_on=['SelectionsNorm','PK_PickListType'], right_on=['SelectionsNorm','PK_PickListType']) 
    dff.rename(columns={'Selections_x': 'Selections'}, inplace=True)
    dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str') 
    dff['PK_Picklist']=dff.ML_Value.astype('int32')
    dff = dff.drop(columns=[c for c in dff.columns if re.search('_\w$', c) ])
    dff['sortorder'] = range(1, len(dff)+1) 
    if 'ZT_Score' in dff.columns:
        dff=dff.drop(columns=['ZT_Score'])
    
    return dff



In [ ]:

def load_sheets():
    dff=pd.DataFrame()
    version='1.5'
    for k in files.keys():
        df = xl_to_df(k, version=version+'\\')
        df = apply_picklists(df)
        dff=pd.concat([df,dff]) 
        
    dff.to_csv(ctx.get_dest() + f'all{version}.csv', index=False)
    dff.to_html(ctx.get_dest() + f'all{version}.html', index=False)

    grp=dff.groupby(['PK_FORM','ID','Question','PK_PickListType'], as_index=False).agg({'Selections': list}) 
    grp.sort_values(['PK_FORM'], inplace=True)
    grp['Selections']=grp['Selections'].apply('|'.join)
    grp.to_html(ctx.get_dest()+ f'all_new_questions{version}.html', escape=False, index=False)
    grp.to_csv(ctx.get_dest() + f'all_new_questions{version}.csv', index=False)

    dff=dff.loc[ dff["SML"].str.contains('^SML(\d)$', na=False, regex=True) ]   

    dff.to_csv(ctx.get_dest() + f'scored{version}.csv', index=False)
    dff.to_html(ctx.get_dest() + f'scored{version}.html', index=False)
load_sheets()


In [ ]:
df15=pd.read_csv(ctx.get_dest() + 'all_new_questions1.5.csv' )  
df15['nQuestion']=df15.Question.apply(qnormalize)
df10=pd.read_csv(ctx.get_dest() + 'all_new_questions1.0.csv' ) 
df10['nQuestion']=df10.Question.apply(qnormalize) 

In [ ]:
df=df15[
    df15['PK_FORM']=='2023-A-HVAMD'
]
df.sort_values('').to_html(ctx.get_dest() + '2023-A-HVAMD.html')

In [ ]:


merged = df10.merge( df15, left_on=[  'ID' ], right_on=[ 'ID' ], how='left', indicator=True, suffixes=('','_y'))
merged=merged.loc[merged._merge=='left_only']
merged=merged.loc[:, ['PK_FORM','ID' ,'Question','PK_PickListType', 'Selections']].drop_duplicates() 

print('|'.join(merged.ID)) 
db=db_parser(ctx).parse('4028:4259')
db=db.loc[db['{idt}'].isin(list(merged.ID)) & db['{idt}'].str.contains('RA')]
db=sort_df_alphanum(db, '{idt}')
code=script_generator(ctx).generate(db)
# print( code ) # did to ID,  todo EP


In [ ]:
def get_new_picks():
    plgen = script_generator(ctx) 
    all=pd.read_csv(ctx.get_dest() + 'all.csv' )  
    dfp=all.loc[all.PK_PickListType==0 ]   
    grp=dfp.groupby('ID', as_index=False).agg({'Selections': list}) 
    grp['PK_PLT'] = [i+600 for i in range(0,len(grp) )]  
    grp['sql']=[plgen.list_to_sql(x, UsageField=y, PK_PickListType=z, PK_PickList=10470, Description=y, encoding=ufencoder)[1] for x,y,z in zip(grp['Selections'], grp['ID'], grp['PK_PLT'])]
    grp['sql']=grp['sql'].apply(lambda s: s.replace('\n','<br>'))
    grp[['sql']].to_html(ctx.get_dest()+'\\newpicks.html', escape=False, index=False)  
    return grp
#new_picks = get_new_picks()
#new_picks 
 

In [ ]:
# pk=get_avail_pk(400, sql="SELECT DISTINCT PK_PickListType FROM vwPicklists") 
# next(pk)

In [ ]:
def db_diff(): 
    dff = pd.read_csv(ctx.get_dest() + 'all_new_questions1.5.csv' )  
    dff['nQuestion']=dff['Question'].apply(qnormalize)
    
    db=db_parser(ctx).parse('4028:4259') 
    db['nQuestion']=db['{QuestionText}'].apply(qnormalize) 
    db.to_html(ctx.get_dest() + 'db.html')
 
    merged = dff.merge(db,  left_on=['nQuestion', 'ID' ], right_on=['nQuestion', '{idt}' ], how='left', indicator=True)
    merged=merged.loc[merged._merge== 'left_only']
    merged=merged.loc[:, ['PK_FORM','ID' ,'Question','PK_PickListType', 'Selections']].drop_duplicates()

    merged['PK_QuestionType']=17  
    merged['sql']="(_PK, @FormName, @PK_QGroup, 1, N'ID', PK_QuestionType, 1, PK_PickListType, 0,NULL,N'Question',NULL),"
    pk=get_avail_pk(42700) 
    qr = question_type_recommender(ctx, verbose=True, use_cache=True)
    for i,r in merged.iterrows():     
        merged.loc[i, 'PK_QuestionType']=qr.recommend(r['Question'])['PK_QuestionType']
        merged.loc[i, 'sql'] = merged.loc[i, 'sql'].replace('_PK',str(next(pk))).replace('ID', merged.loc[i, 'ID']).replace('PK_PickListType', str(merged.loc[i, 'PK_PickListType'])).replace('PK_QuestionType', str(merged.loc[i, 'PK_QuestionType'])).replace('Question', merged.loc[i, 'Question'])
  
    return merged
dbdiff=db_diff()
dbdiff


In [ ]:
 
for pkform in [k for k in pkforms.keys() if k =='2023-A-HVAARC']:
    print(pkform)
    script = dbupdate_parser(ctx, path=pkforms[pkform], verbose=True)   
    form = dbdiff.loc[dbdiff.PK_FORM == pkform ]
    for i,r in form.iterrows(): 
        #script.update_question_text( r['ID'], r['Question'].replace("'","''") )  
        insert = script.get_insert(r['ID'])
    #script.inspect()
    print(script.nf) 
dbdiff.loc[dbdiff['ID'].isin(script.nf)]['sql'].to_csv(ctx.get_dest() + 'sql.csv', index=False)
 

In [ ]:
scored = pd.read_csv(f'{ctx.get_dest()}scored1.5.csv').fillna('')
scored = scored.drop(['nQuestion', 'Question', 'SelectionsNorm'], axis=1)
scored['Selections'] = scored['Selections'].apply(lambda s: s[:10])
cols=['PK_FORM','ID', 'SML', 'PK_Picklist'] 
for c in [c for c in scored.columns if not re.search( 'PK_FORM|SML|^ID$|PK_Picklist$|Basic_|Mod_|Adv_' ,c)]:
    cols.append(c)
print(cols)
scored=scored[cols]
ins, create, temp=SQL_INSERT_FROM_DF(scored, TABLE_NAME='HVASAScoring') 
script = '\n'.join(ins)
with open(f'{ctx.get_dest()}script.sql', 'w') as f: 
    f.write(f"{create} \n\n {script}"  )
with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\SA\_101_DB_Update_SAScoring_Excel_Import.sql', 'w') as f: 
    f.write( f"{create} \n\n {script}"  )
 